# ♎ IC Passives

In this notebook, we'll cover passive components such as resistors and capacitors in IC design. We'll explore how these components are made and operate, both within the silicon substrate and the metallization layers added during the IC manufacturing process.

🎥 Video Insight:

<iframe width="560" height="315" src="https://www.youtube.com/embed/nhPX8IFYk9Y?si=-4F-BuWoEnhkpoKp" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

Now, let's roll up our sleeves and dive into the SKY130 PDK to see how these components manifest themselves!

### Polysilicon/Metal Resistors

#### Resistivity

Resistors are often crafted from silicide-doped polysilicon or metal interconnects. Their behavior is majorly governed by the resistance per unit ratio parameter, $R_{\square}$, with the actual resistance determined by:
$$R = \dfrac{R_{\square}L}{W}$$

Here's a handy table showcasing the $R_{\square}$ values for various polysilicon/metal resistors in the SKY130 PDK:

| **Material**       | **PDK Device Name**              | **Hdl21 Name** | **Nominal $R_{\square}$** |
| ------------------ | ---------------------------- | -------------- | ------------------------- |
| N+ doped gate poly | sky130_fd_pr__res_generic_po | GEN_PO         | $48.2\Omega/\square$      |
| Local Interconnect | sky130_fd_pr__res_generic_l1 | GEN_L1         | $12.8\Omega/\square$      |
| Metal 1            | sky130_fd_pr__res_generic_m1 | GEN_M1         | $0.125\Omega/\square$     |
| Metal 2            | sky130_fd_pr__res_generic_m2 | GEN_M2         | $0.125\Omega/\square$     |
| Metal 3            | sky130_fd_pr__res_generic_m3 | GEN_M3         | $0.047\Omega/\square$     |
| Metal 4            | sky130_fd_pr__res_generic_m4 | GEN_M4         | $0.047\Omega/\square$     |
| Metal 5            | sky130_fd_pr__res_generic_m5 | GEN_M5         | $0.0285\Omega/\square$    |

While these "generic" resistors are great for modeling layouts, they might not be the best choice for analog designs. Instead, the SKY130 wizards recommend using "precision" P-doped polysilicon resistors with fixed widths. Let's delve into their parameters:

Note: These parameters don't behave like a naive sheet resistor, as they are actually slot local interconnects of a fixed length connected to either side of a silicide-doped poly sheet. So the equation of resistance is given by:

$$R = \dfrac{R_0}{W} + \dfrac{R_\square L}{W}$$ 

Where $R_\square$ is the usual resistance per unit ratio and $R_0$ represents the slot local interconnect sheet resistance, which is assumed to have fixed length.

| **Material** | **PDK Device Name** | **Hdl21 Name** | **Nominal $R_{\square}$** | **Nominal $R_0$** |
| ------------ | --------------- | -------------- | ------------------------- | ----------------- |
| P+ doped poly| sky130_fd_pr__res_high_po_xxxx | PP_PREC_xxxx   | $378\Omega/\square$   | $254.2 \mu\Omega m$  |
| P- doped poly| sky130_fd_pr__res_xhigh_po_xxxx| PM_PREC_xxxx   | $2080.3\Omega/\square$ | $199.5 \mu\Omega m$ |

In the below simulation, we compute the values in this table exactly, to show how these devices can be characterized from simulations.

##### Exercises:

1. Investigate SKY130 Generic resistors, can you confirm their sheet resistivity?
2. Studying the datasheet or SPICE files of SKY130 Generic resistors, why might they be inappropriate for analog designs?
3. Experiment with the precision resistor sheet resistivity calculations - what are acceptable bounds on $R_\square$ and $R_0$?

In [6]:
import hdl21 as h
import sky130_hdl21.primitives as p
import sky130_hdl21 as sky130
import vlsirtools.spice as vsp
from sitepdks import *

prp = p.Sky130PrecResParams

# We will be using the 5 different precision resistors
pp_prec_res = [p.PP_PREC_0p35,p.PP_PREC_0p69,p.PP_PREC_1p41,p.PP_PREC_2p85,p.PP_PREC_5p73]
pm_prec_res = [p.PM_PREC_0p35,p.PM_PREC_0p69,p.PM_PREC_1p41,p.PM_PREC_2p85,p.PM_PREC_5p73]

def measure_resistance(prec_res, width, length):
    
    @h.sim.sim
    class ResistanceSim:

        @h.module
        class Tb:

            VSS = h.Port()
            VDD = h.Signal()

            vdd = h.DcVoltageSource(dc=1)(p=VDD, n=VSS)
            res = prec_res(l=length)(p=VDD, n=VSS, b=VSS)

        # Simulation Controls
        op = h.sim.Op()
        inc = sky130.install.include(h.pdk.Corner.TYP)

    # Run the simulation
    opts = vsp.SimOptions(
        simulator=vsp.SupportedSimulators.NGSPICE,
        fmt=vsp.ResultFormat.SIM_DATA,
        rundir="./scratch",
    )
    rv = ResistanceSim.run(opts)
    op = rv[vsp.sim_data.AnalysisType.OP]

    # Use Ohm's law to calculate resistance
    return 1/-op.data['i(v.xtop.vvdd)']

# We use a substition on R = R0/W + Rsq*L/W
# with y=R, R0=m, x=1/W, Rsq*L/W=b
# Converting our equation into y=mx+b form
# We can then use linear regression to determine m and b
import numpy as np

def determine_parameters(width_list,resistance_list):

    x = 1/np.array(width_list)
    y = np.array(resistance_list)

    m,b = np.polyfit(x,y,1)

    return m,b

# We now iterate through all the different precision resistors
# being careful to set W = L so that b = RsqL/W = Rsq
width_list = [0.35,0.69,1.41,2.85,5.73]
resistance_list = []

for idx,w in enumerate(width_list):
    resistance_list.append(measure_resistance(pp_prec_res[idx],w,w))

# We now determine the parameters for the linear regression
m,b = determine_parameters(width_list,resistance_list)
print(f"P+ doped poly resistor parameters: R0 = {m}, Rsq = {b}")

# Repeat for the P- doped poly resistors
resistance_list = []
for idx,w in enumerate(width_list):
    resistance_list.append(measure_resistance(pm_prec_res[idx],w,w))

m,b = determine_parameters(width_list,resistance_list)
print(f"P- doped poly resistor parameters: R0 = {m}, Rsq = {b}")

P+ doped poly resistor parameters: R0 = 254.21359097420188, Rsq = 377.9731755794322
P- doped poly resistor parameters: R0 = 199.48633578280175, Rsq = 2080.287031426221


#### Parasitic Capacitance

While the nominal resistivity of our resistors is probably the most interesting parameter to us - every integrated device is also secretly a capacitor and it's important for us to be able to detect and know the approximate parameters of this parasitic capacitance. The parasitic capacitance of a poly resistor is that of a sheet capacitor, with the poly acting as a plate on the top and the silicon beneath acting as the plate on the bottom, ie.

$$C = C_0WL$$

So, we can measure the parasitic capacitance by looking at frequency response of resistor to a signal across the two "ordinary" terminals of the resistor and ground to get the results:

| **Material** | **PDK Device Name** | **Hdl21 Name** | **Parasitic Sheet Capacitance $C_0$** |
| ------------ | --------------- | -------------- | ------------------------- | 
| P+ doped poly| sky130_fd_pr__res_high_po_xxxx | PP_PREC_xxxx   | $1.4 fF/\mu m^2$   | 
| P- doped poly| sky130_fd_pr__res_xhigh_po_xxxx| PM_PREC_xxxx   | $1.3 fF/\mu m^2$ |

##### Exercises

1. Simulate a circuit with a poly resistor and an AC source. Measure the frequency response and determine the point at which the parasitic capacitance starts to significantly affect the circuit's behavior.
2. Research and discuss real-world scenarios where parasitic capacitance in poly resistors has caused issues in circuit performance. How were these issues mitigated?
3. Explore other materials or resistor types in the PDK. How do their parasitic capacitances compare to poly resistors?

In [48]:
def measure_capacitance(prec_res, width, length):
    
    @h.sim.sim
    class CapacitanceSim:

        @h.module
        class Tb:

            VSS = h.Port()
            VDD = h.Signal()

            vdd = h.DcVoltageSource(ac=1)(p=VDD, n=VSS)
            res = prec_res(l=length)(p=VDD, n=VDD, b=VSS)

        # Simulation Controls
        ac = h.sim.Ac(sweep=h.sim.LogSweep(1e1, 1e10, 10))
        inc = sky130.install.include(h.pdk.Corner.TYP)

    # Run the simulation
    opts = vsp.SimOptions(
        simulator=vsp.SupportedSimulators.NGSPICE,
        fmt=vsp.ResultFormat.SIM_DATA,
        rundir="./scratch",
    )
    rv = CapacitanceSim.run(opts)
    op = rv[vsp.sim_data.AnalysisType.AC]

    # This piece of math returns the capacitance
    omega = np.array([2*np.pi*10**(1+i/10) for i in range(91)])
    return np.mean(-op.data['i(v.xtop.vvdd)']/(1j*omega))

# We now iterate through all the different precision resistors
capacitance_list = []
for idx,w in enumerate(width_list):
    capacitance_list.append(measure_capacitance(pp_prec_res[idx],w,w))

# We now determine the parameters the sheet capacitance by fitting a linear regression
m,b = np.polyfit(width_list,capacitance_list,1)
print(f"P+ doped poly resistor sheet capacitance: C0 = {m.real}")

# Repeat for the P- doped poly resistors
capacitance_list = []
for idx,w in enumerate(width_list):
    capacitance_list.append(measure_capacitance(pm_prec_res[idx],w,w))

# We now determine the parameters the sheet capacitance by fitting a linear regression
m,b = np.polyfit(width_list,capacitance_list,1)
print(f"P- doped poly resistor sheet capacitance: C0 = {m.real}")

P+ doped poly resistor sheet capacitance: C0 = 1.4076044815178863e-15
P- doped poly resistor sheet capacitance: C0 = 1.2959022151888223e-15
